In [21]:
import datumaro as dm
import os
from os import path as osp
import cv2
from matplotlib import pyplot as plt
import glob
from ultralytics import YOLO
from ultralytics import settings
from abc import ABC, abstractmethod
from pathlib import Path

from collections import defaultdict
import numpy as np

# Update a setting
settings.update({'runs_dir': '/training/runs'})

DUP_LABELS_MAPPING = {
        'White Fish': 'Whitefish',
        'Bull Trout': 'Bull',
        'Lan prey': 'Lamprey',
        'Lampray': 'Lamprey'
        }

# DataLoader
* data_frames
* load()
* next_frame()
# SalmonCounter
* DataLoader field
* model
* track_history
* LINE_OF_INTEREST_RATIO
* load()
* count()
# Evaluator
* evaluate()

In [ ]:
def track_img_frames(model_path, frames_folder):
    botsort='botsort.yaml'
    bytetrack='bytetrack.yaml'
    
    model = YOLO(model_path)
    frames = glob.glob(osp.join(frames_folder, '*.jpg'))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (1920, 1080))
    for frame in sorted(frames):
        results = model.track(frame, persist=True, tracker=bytetrack)
        
        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        out.write(annotated_frame)

    out.release()

track_img_frames('runs/detect/train20/weights/best.pt', '/mnt/shiorissd4tb/masamim/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/')
#track_img_frames('train_57_no_filt/weights/best.pt', '/home/masamim/salmon-computer-vision/utils/export_bear_creek_all_yolo/test/09-04-2021_06-42-05_m_network_camera/obj_train_data/')

In [ ]:
class DataLoader(ABC):
    @abstractmethod
    def frames():
        pass

class ImageDirLoader(DataLoader):
    IMG_PATTERN = '*.[jpJP][npNP]*[gG$]'
    def __init__(self, img_folder):
        path = Path(img_folder).resolve()
        if not path.is_dir():
            raise ValueError(f"'{path}' is not a directory.")
        self.image_dir = path
        
    def frames(self):
        frec = lambda p, g: p.glob(g)
        for f in frec(self.image_dir, self.IMG_PATTERN):
            yield f

d = ImageDirLoader('/mnt/shiorissd4tb/masamim/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/')
for x in d.frames():
    print(x)

In [ ]:
class SalmonCounter:
    def __init__(self, model_path: str, dataloader: DataLoader):
        self.model = YOLO(model_path)
        self.dataloader = dataloader
        self.track_history = defaultdict(lambda: [])
        self.salm_count = defaultdict(lambda: [])

    def count():
        for frame in self.dataloader.frames():
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True)
    
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
    
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
    
            # TODO: Implement LOI alg to count
            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = self.track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                #if len(track) > 30:  # retain 90 tracks for 90 frames
                #    track.pop(0)
    
                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

    def _line_of_interest(self, f_width, track_id, track):
        # Remember currently tracking IDs
        # Once a track disappears for maybe 3 frames
        return